## Heat equation in 5 dimensions

Comparing PINN solution to exact solution.

In [1]:
import torch
from torch import sin as sin
from model_architecture import PINN
pi = 3.1415926535898


def exact_solution(x_list, t):
    return sin(pi*x_list[0])*sin(2*pi*x_list[1])*sin(pi*x_list[2])*sin(pi*x_list[3])*sin(pi*x_list[4])*torch.exp(-8*alpha*pi**2*t) \
        + sin(2*pi*x_list[0])*sin(pi*x_list[1])  *sin(2*pi*x_list[2])*sin(pi*x_list[3])*sin(pi*x_list[4])*torch.exp(-11*alpha*pi**2*t)

alpha = 0.04  # Thermal diffusivity
spatial_dim = 5

# Load PINN model
model = PINN(input_dim=spatial_dim+1)
model.load_state_dict(torch.load('weights1_5D.pth'))

model.eval()
x1 = torch.linspace(0, 1, 10)
x2 = torch.linspace(0, 1, 10)
x3 = torch.linspace(0, 1, 10)
x4 = torch.linspace(0, 1, 10)
x5 = torch.linspace(0, 1, 10)
times = torch.tensor([0.0, 0.5, 1.0])

X1, X2, X3, X4, X5 = torch.meshgrid(x1, x2, x3, x4, x5, indexing='ij')

for i, t in enumerate(times):
    T = t * torch.ones_like(X1)
    with torch.no_grad():
        U = model([X1.reshape(-1, 1), X2.reshape(-1, 1), X3.reshape(-1, 1), X4.reshape(-1, 1), X5.reshape(-1, 1)], T.reshape(-1, 1)).reshape(X1.shape)

    U_exact = exact_solution([X1, X2, X3, X4, X5], T)
    
    abs_diff = torch.abs(U - U_exact)
    max_exact = torch.max(torch.abs(U_exact))
    diff_ratio = abs_diff / max_exact

    print('\nt = ', t.item())
    print('Max diff ratio:', torch.max(diff_ratio).item())
    print('Mean diff ratio:', torch.mean(diff_ratio).item())
    print('Median diff ratio:', torch.median(diff_ratio).item())


t =  0.0
Max diff ratio: 1.2954634428024292
Mean diff ratio: 0.05728160962462425
Median diff ratio: 4.529211672377187e-09

t =  0.5
Max diff ratio: 1.305808424949646
Mean diff ratio: 0.060437850654125214
Median diff ratio: 3.3104847751985744e-09

t =  1.0
Max diff ratio: 1.1719342470169067
Mean diff ratio: 0.06301143020391464
Median diff ratio: 7.633872201040504e-09
